# Locate Stations
Each record in the 1% data set contains a start an stop location for the bike.  Using averaging this data I am creating a location for the station.  This way when plotting rides the data point will plot to the same spot rather than creating a "fuzy cloud" where the station is located.
* Calculate (longitudes and latitudes) for each station base on average for all the records that either started on ended at that station
* Create a CSV file of station locations.  Each record has Station_ID, lat, and Lng
* This csv / data frame will be joined with 1% files createing a new variable for the start stop location.
* THIS DATA WAS COMBINED USING AN EXCEL FILE AND XLOOKUP

In [6]:
import os
import pandas as pd

---
### Load the 1% data file

In [7]:
# file_one = "Resources/DataOne.csv"
file_one = os.path.join('.','Data', 'New York Data','New York Data 1Prct', 'City_Bike_2022_TripData_augmented_1P.csv')            # In this example my Jupyter notebook is in the current directory " . "
                                                                    # There is a sub directory "Resources" and a file there "Olympics.csv"
bddf = pd.read_csv(file_one, encoding="utf-8")                     # Opens the CVS file and read it in as a panda dataframe 


C:\Users\jspinega\AppData\Local\Temp\ipykernel_14384\377296711.py:4: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  bddf = pd.read_csv(file_one, encoding="utf-8")                     # Opens the CVS file and read it in as a panda dataframe


In [3]:
bddf.head()


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,duration,start_DOW,end_DOW,round_trip
0,9D145B6BF31BAA44,electric_bike,2022-01-19 15:29:15,2022-01-19 15:50:38,Tompkins Ave & Hopkins St,4850.04,E 20 St & 2 Ave,5971.08,40.699576,-73.947084,40.735877,-73.982050,member,1283,Wednesday,Wednesday,False
1,D85648D425D3A941,electric_bike,2022-01-26 16:38:14,2022-01-26 16:48:46,6 Ave & 12 St,3803.05,Grand Ave & Bergen St,4190.06,40.666318,-73.985462,40.678045,-73.962408,member,632,Wednesday,Wednesday,False
2,526FF90AC12FF6F1,classic_bike,2022-01-03 08:31:58,2022-01-03 08:34:57,W 42 St & 6 Ave,6517.08,W 47 St & 6 Ave,6667.04,40.754920,-73.984550,40.758397,-73.982550,member,179,Monday,Monday,False
3,E661187876A5E48D,classic_bike,2022-01-27 12:50:06,2022-01-27 13:12:56,Kent Ave & S 11 St,5062.01,St James Pl & Oliver St,5238.05,40.707645,-73.968415,40.713079,-73.998512,member,1370,Thursday,Thursday,False
4,44A97975491D1AF6,classic_bike,2022-01-28 13:36:51,2022-01-28 13:46:33,Lafayette St & Jersey St,5561.06,1 Ave & E 16 St,5779.08,40.724561,-73.995653,40.732219,-73.981656,member,582,Friday,Friday,False


In [17]:
# create a data frame of only the starting locations with their longitudes and latitudes
start_df = bddf[['start_station_id','start_lat', 'start_lng']]

In [18]:
# create a data frame of only the ending locations with their longitudes and latitudes
end_df = bddf[['end_station_id','end_lat', 'end_lng']]

In [19]:
# rename the column headings
station1_df = start_df.rename(columns={'start_station_id': 'station_ID','start_lat':'station_lat', 'start_lng':'station_log'})
station2_df = end_df.rename(columns={'end_station_id': 'station_ID','end_lat':'station_lat', 'end_lng':'station_log'})

In [20]:
# merge the starting and ending station into a single data frame
merge_df = station1_df.append(station2_df)

C:\Users\jspinega\AppData\Local\Temp\ipykernel_14384\1649270471.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merge_df = station1_df.append(station2_df)


In [21]:
merge_df.head()


,station_ID,station_lat,station_log
0,4850.04,40.699576,-73.947084
1,3803.05,40.666318,-73.985462
2,6517.08,40.754920,-73.984550
3,5062.01,40.707645,-73.968415
4,5561.06,40.724561,-73.995653


In [24]:
# Using Groupby determine calculate the average longitudes and latitudes for each station using both the starting records and ending records
groupby_bdf =  merge_df.groupby(['station_ID'])
station_lat = groupby_bdf['station_lat'].mean()
station_log = groupby_bdf['station_log'].mean()
station_loc = pd.DataFrame({'lat':station_lat,'lng':station_log})
station_loc.head()

,lat,lng
station_ID,,
2733.03,40.633387,-74.016559
2782.02,40.635747,-74.019941
2821.05,40.635562,-74.012986
2832.03,40.637020,-74.022152
2872.02,40.637669,-74.017818


In [26]:
station_loc.info()


<class 'pandas.core.frame.DataFrame'>
Index: 3530 entries, 2733.03 to SYS038
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   lat     3530 non-null   float64
 1   lng     3530 non-null   float64
dtypes: float64(2)
memory usage: 82.7+ KB


In [28]:
# create a CSV file that has a single longitude and latitude for each station ID.
file_one = os.path.join('.','Data', 'New York Data','New York Data 1Prct', 'City_Bike_station_location.csv') 
station_loc.to_csv(file_one, index=True, header=True)      